In [31]:
try: 
    import seaborn as sns
except:
    !pip install seaborn
    import seaborn as sns
try:
    import pymc as pm # For MCMC
except:
    !pip install pymc
    import pymc as pm
try:
    import arviz as az # For MCMC package
except:
    !pip install arviz
    import arviz as az
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
try:
    import corner
except:
    !pip install corner
    import corner

print(pm.__version__)

5.15.0


In [32]:
def dfg(a,b):
    file_path = 'datasets/Donor%d_CD%d_Genes.csv' %(a,b)
    all_df = pd.read_csv(file_path)
    file_path = 'datasets/mt_genes_metadata.csv'
    dfmeta = pd.read_csv(file_path)
    protein_coding_genes = dfmeta[dfmeta['gene_type'] == 'protein_coding']
    protein_names = protein_coding_genes['gene_name'].tolist()
    df2 = pd.DataFrame([all_df[i] for i in protein_names]).T
    return df2

In [33]:
def normiz_2(df,s_n = 1000):
    df1 = df
    listfinal = [sum(df.iloc[i]) for i in range(len(df))]
    for i in range(len(df)):
        df1.iloc[i] = df1.iloc[i]/listfinal[i] * s_n
    return df1

In [34]:
def g_log(df_n):
    df_final = normiz_2(df_n).T
    gene_mean = [np.mean(df_final.iloc[i]) for i in range(13)]
    gene_var = [np.var(df_final.iloc[i]) for i in range(13)]
    log_gene_mean = np.log(gene_mean)
    log_gene_var = np.log(gene_var)
    return log_gene_mean, log_gene_var

def log_gene_plot(df_n):
    df_final = normiz_2(df_n).T
    gene_mean = [np.mean(df_final.iloc[i]) for i in range(13)]
    gene_var = [np.var(df_final.iloc[i]) for i in range(13)]
    log_gene_mean = np.log(gene_mean)
    log_gene_var = np.log(gene_var)
    plt.scatter(log_gene_mean,log_gene_var)

In [35]:
def lse(a,b):
    x = g_log(normiz_2(dfg(a,b)))[0]
    y = g_log(normiz_2(dfg(a,b)))[1]

    X = np.vstack([np.ones(len(x)), x]).T
    # theta = (X^T X)^(-1) X^T y
    theta = np.linalg.inv(X.T @ X) @ X.T @ y

    a, b = theta
    print(f"Estimated parameters: a = {a}, b = {b}")

    y_pred = a + b * x

    ss_total = np.sum((y - np.mean(y))**2)
    ss_residual = np.sum((y - y_pred)**2)
    r_squared = 1 - (ss_residual / ss_total)

    print(f"R-squared (R^2) value: {r_squared}")

    plt.scatter(x, y, label='Observed data')
    plt.plot(x, y_pred, color='red', label='Fitted line')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Least Squares Estimation')
    plt.legend()
    plt.show()

---

In [53]:
import numpy as np
import pandas as pd
from scipy.stats import nbinom, chisquare
def negative_binomial_gof(data, bins=None):
    """
    Perform a goodness-of-fit test to check if the data follows a negative binomial distribution.
    
    Parameters:
    - data: array-like, the observed data to test
    - bins: int or None, number of bins to group the data into for stabilization
    
    Returns:
    - chi_square_stat: float, the chi-square statistic
    - p_value: float, the p-value of the chi-square test
    - r: float, estimated parameter r (number of successes)
    - p: float, estimated parameter p (probability of success)
    """
    # Ensure data is a numpy array
    data = np.array(data)
    
    # Step 1: Calculate mean and variance of the data
    mean = np.mean(data)
    variance = np.var(data)
    
    # Step 2: Estimate the parameters r and p
    if variance <= mean:
        raise ValueError("Variance must be greater than the mean for a negative binomial distribution.")
    
    r = mean**2 / (variance - mean)
    p = mean / variance
    print(f"Estimated parameters: r = {r}, p = {p}")
    
    # Step 3: Create observed frequency table
    if bins is None:
        observed_freq = pd.Series(data).value_counts().sort_index()
    else:
        observed_freq, bin_edges = np.histogram(data, bins=bins)
        observed_freq = pd.Series(observed_freq, index=bin_edges[:-1])
    
    # Step 4: Calculate expected frequencies
    expected_freq = []
    for k in observed_freq.index:
        prob = nbinom.pmf(k, r, p)
        expected_freq.append(prob * len(data))
    
    # Convert to numpy arrays for the chi-square test
    observed_freq = observed_freq.values
    expected_freq = np.array(expected_freq)
    
    # Normalize expected frequencies to sum to observed frequencies
    expected_freq_sum = expected_freq.sum()
    observed_freq_sum = observed_freq.sum()
    expected_freq = expected_freq * (observed_freq_sum / expected_freq_sum)
    
    # Ensure no zero expected frequencies
    expected_freq = np.where(expected_freq == 0, 1e-10, expected_freq)
    
    print("Observed frequencies:\n", observed_freq)
    print("Expected frequencies:\n", expected_freq)
    
    # Step 5: Perform the chi-square goodness of fit test
    chi_square_stat, p_value = chisquare(f_obs=observed_freq, f_exp=expected_freq)
    
    # Output results
    print(f"Chi-Square Statistic: {chi_square_stat}")
    print(f"p-value: {p_value}")
    
    # Interpretation
    if p_value < 0.05:
        print("Reject the null hypothesis: The data does not follow a negative binomial distribution.")
    else:
        print("Fail to reject the null hypothesis: The data could follow a negative binomial distribution.")
    
    return chi_square_stat, p_value, r, p

In [55]:
negative_binomial_gof(dfg(1,4)["MT-ND1"])

Estimated parameters: r = 3.619771220237507, p = 0.3161248033764886
Observed frequencies:
 [ 54 101 161 200 250 260 293 253 250 209 155 143 120  91  71  50  56  37
  31  30  16  12   9  10   6   3   4   2   2   2   2   1   1   1   1   1]
Expected frequencies:
 [4.47119735e+01 1.10683228e+02 1.74843359e+02 2.23987290e+02
 2.53503066e+02 2.64199503e+02 2.59569308e+02 2.43947794e+02
 2.21461849e+02 1.95537853e+02 1.68755982e+02 1.42893725e+02
 1.19055466e+02 9.78267236e+01 7.94202714e+01 6.37994927e+01
 5.07748251e+01 4.00747024e+01 3.13948560e+01 2.44305274e+01
 1.88959141e+01 1.45345353e+01 1.11234456e+01 8.47351495e+00
 6.42737338e+00 4.85613163e+00 3.65561216e+00 2.74255219e+00
 2.05104724e+00 1.52937238e+00 1.13723378e+00 8.43450623e-01
 6.24037562e-01 1.33990631e-01 9.79501624e-02 2.04436265e-03]
Chi-Square Statistic: 527.9836959516022
p-value: 2.5194348890031197e-89
Reject the null hypothesis: The data does not follow a negative binomial distribution.


(527.9836959516022,
 2.5194348890031197e-89,
 3.619771220237507,
 0.3161248033764886)

In [50]:
np.mean(normiz_2(dfg(1,4))["MT-ND1"])

48.12274271988561